# Model of Scientific Discovery

This notebook provides analysis for the Model of Scientific Discovery by using recurrence relations.

In [ ]:
import matplotlib.pyplot as plt

## Parameters

First we need to set up starting values the following model parameters:

| parameter | description | permissible values |
| --- | --- | --- |
| $a$ | number of agents | integral, $n > 0$ |
| $r$ | probability that an agent chooses to replicate a tested hypothesis | $0 \leq r \leq 1$ |
| $b$ | probability that a novel hypothesis is true | $0 \leq b \leq 1$ |
| $\alpha$ | type I error rate | $0 \leq \alpha \leq 1$ |
| $\beta$ | type II error rate | $0 \leq \beta \leq 1$ |
| $c_{N+}$ | probability that a positive novel result is published | $0 \leq c_{N+} \leq 1$ |
| $c_{N-}$ | probability that a negative novel result is published | $0 \leq c_{N-} \leq 1$ |
| $c_{R+}$ | probability that a positive replication result is published | $0 \leq c_{R+} \leq 1$ |
| $c_{R-}$ | probability that a negative replication result is published | $0 \leq c_{R-} \leq 1$ |

These will be changed in examples below, but for now we just want a complete set to work with.

In [ ]:
a = 100
r = 0.2
b = 0.001
alpha = 0.05
beta = 0.2
c_n_pos = 1.0
c_n_neg = 1.0
c_r_pos = 0.9
c_r_neg = 0.9

## Setting up data structures

Here we will make a simplifying assumption that the tally numbers never exceed some minimum and maximum value. We make this assumption to simplify the code to gather the results. Note that this assumption is reasonable: very few hypotheses will exceed the minimum and maximum values provided that these values are extreme enough.

For reasons that should hopefully be obvious, we require $$\text{max_tally} > 1$$ and $$\text{min_tally} < -1.$$

In [ ]:
max_tally = 1000
min_tally = -1000

Next we set up arrays to keep track of $n_{\text{T}, s}$ and $n_{\text{F}, s}$ values, as described in the main text. Note that keeping track of these values for all time steps is trivial. However, we will not bother doing that here.

In [ ]:
n_Ts = [0] * (max_tally - min_tally + 1)
n_Fs = [0] * (max_tally - min_tally + 1)

# The total number of hypotheses
n = 0

# This is a function to convert tally numbers to indices in the above
# array
tally_idx = lambda x: x - min_tally

## Defining the recurrence relations

Next we define functions for the recursion relations, taking special care to handle the cases where $s = 1, -1, \text{min_tally}, \text{max_tally}$.

In [ ]:
published_failed_true_replications_from_above = (
    lambda s: a * r * n_Ts[tally_idx(s + 1)] / n * beta * c_r_neg
)
published_successful_true_replications_from_below = (
    lambda s: a * r * n_Ts[tally_idx(s - 1)] / n * (1 - beta) * c_r_pos
)
published_true_replications_from_current = (
    lambda s: a * r * n_Ts[tally_idx(s)] / n * ((1 - beta) * c_r_pos + beta * c_r_neg)
)
new_positive_true_novel_hypotheses = a * (1 - r) * b * (1 - beta) * c_n_pos
new_negative_true_novel_hypotheses = a * (1 - r) * b * beta * c_n_neg


def run_recursion_for_true_hypotheses(s):
    global n

    # Get number of true hypotheses for this tally from previous time
    # step
    count = n_Ts[tally_idx(s)]

    # Add in failed replications from above
    if s != max_tally:
        count += published_failed_true_replications_from_above(s)

    # Add in successful replications from below
    if s != min_tally:
        count += published_successful_true_replications_from_below(s)

    # Remove in replications of current tally
    count -= published_true_replications_from_current(s)

    # Add in new novel hypotheses
    if s == 1:
        count += new_positive_true_novel_hypotheses
        n += new_positive_true_novel_hypotheses
    elif s == -1:
        count += new_negative_true_novel_hypotheses
        n += new_negative_true_novel_hypotheses

    return count

In [ ]:
published_failed_false_replications_from_above = (
    lambda s: a * r * n_Fs[tally_idx(s + 1)] / n * (1 - alpha) * c_r_neg
)
published_successful_false_replications_from_below = (
    lambda s: a * r * n_Fs[tally_idx(s - 1)] / n * alpha * c_r_pos
)
published_false_replications_from_current = (
    lambda s: a * r * n_Fs[tally_idx(s)] / n * (alpha * c_r_pos + (1 - alpha) * c_r_neg)
)
new_positive_false_novel_hypotheses = a * (1 - r) * (1 - b) * alpha * c_n_pos
new_negative_false_novel_hypotheses = a * (1 - r) * (1 - b) * (1 - alpha) * c_n_neg


def run_recursion_for_false_hypotheses(s):
    global n

    # Get number of false hypotheses for this tally from previous time
    # step
    count = n_Fs[tally_idx(s)]

    # Add in failed replications from above
    if s != max_tally:
        count += published_failed_false_replications_from_above(s)

    # Add in successful replications from below
    if s != min_tally:
        count += published_successful_false_replications_from_below(s)

    # Remove in replications of current tally
    count -= published_false_replications_from_current(s)

    # Add in new novel hypotheses
    if s == 1:
        count += new_positive_false_novel_hypotheses
        n += new_positive_false_novel_hypotheses
    elif s == -1:
        count += new_negative_false_novel_hypotheses
        n += new_negative_false_novel_hypotheses

    return count

## Setting up initial data

We need to set up at least one hypothesis to get the model started. Here we'll set up $50$ true and false hypotheses with tally $1$ and $50$ true and false hypotheses with tally $-1$.

In [ ]:
n_Ts[tally_idx(1)] = 50
n_Fs[tally_idx(1)] = 50
n_Ts[tally_idx(-1)] = 50
n_Ts[tally_idx(-1)] = 50

n = 200

## Running the recurrence relations

We now run the recurrence relations for a fixed number of time steps.

In [ ]:
num_time_steps = 500

for _ in range(num_time_steps):
    new_n_Ts = n_Ts.copy()
    new_n_Fs = n_Fs.copy()

    for s in range(min_tally, max_tally):
        new_n_Ts[tally_idx(s)] = run_recursion_for_true_hypotheses(s)
        new_n_Fs[tally_idx(s)] = run_recursion_for_false_hypotheses(s)


    n_Ts = new_n_Ts
    n_Fs = new_n_Fs

In [ ]:
plt.bar(list(range(len(n_Ts))), [x / n for x in n_Ts])


In [ ]:
plt.bar(list(range(len(n_Fs))), [x / n for x in n_Fs])

In [ ]:
print(max(n_Fs))